In [17]:
from pathlib import Path
import shutil
import pandas as pd
import warnings
import gc

warnings.filterwarnings('ignore')

In [18]:
DATA_STORE = Path('E:/mlft/data/assets.h5')
print(DATA_STORE.cwd())

E:\mlft\data


In [19]:
with pd.HDFStore(DATA_STORE, mode='r') as store:
    all_keys = store.keys()       
all_keys

['/engineered_features',
 '/us/snp500/tickers/fdr',
 '/us/snp500/prices/yfinance',
 '/us/nasdaq/tickers/fdr',
 '/us/nasdaq/prices/yfinance',
 '/us/etfs/tickers/etfdb',
 '/us/etfs/prices/yfinance',
 '/stooq/us/nysemkt/stocks/prices',
 '/stooq/us/nysemkt/stocks/tickers',
 '/stooq/us/nyse/stocks/prices',
 '/stooq/us/nyse/stocks/tickers',
 '/stooq/us/nyse/etfs/prices',
 '/stooq/us/nyse/etfs/tickers',
 '/stooq/us/nasdaq/stocks/prices',
 '/stooq/us/nasdaq/stocks/tickers',
 '/stooq/us/nasdaq/etfs/prices',
 '/stooq/us/nasdaq/etfs/tickers',
 '/stooq/jp/tse/stocks/prices',
 '/stooq/jp/tse/stocks/tickers',
 '/sp500/fred',
 '/sp500/stocks',
 '/sp500/stooq',
 '/quandl/wiki/prices',
 '/quandl/wiki/stocks',
 '/kr/etf/tickers/fdr',
 '/kr/etf/prices/fdr',
 '/fred/assets']

### data to zipline bundle

In [20]:
price_keys = [key for key in all_keys if 'prices' in key and'etfs' in key and 'yfinance' in key]    

In [21]:
price_keys

['/us/etfs/prices/yfinance']

In [26]:
def clear_directory(directory):
    """디렉토리와 그 안의 모든 파일 및 서브디렉토리를 삭제합니다."""
    directory = Path(directory)
    if directory.exists() and directory.is_dir():
        shutil.rmtree(directory)  # 디렉토리와 그 안의 모든 파일/서브디렉토리 삭제
    directory.mkdir(parents=True, exist_ok=True)  # 디렉토리 다시 생성


def process_and_save_chunk(data_store, key, start, stop, output_folder):

    
    with pd.HDFStore(data_store, mode='r') as store:
        chunk = store.select(key, start=start, stop=stop)
        
        for ticker in chunk.index.get_level_values('ticker').unique():
            df_price = chunk.xs(ticker, level='ticker')
            output_file = output_folder / f'{ticker}.csv'
            
            if start == 0 and output_file.exists():
                # 첫 번째 청크인 경우 기존 파일 삭제
                output_file.unlink()
            
            # CSV 파일에 다중 인덱스를 유지한 채로 저장
            df_price.to_csv(output_file, mode='a', header=not output_file.exists())


In [27]:
chunk_size = 100000  # 조절 가능한 청크 크기
for price_key in price_keys:
    key_splited = price_key.split('/')
    output_folder = f"{key_splited[1]}_{key_splited[2]}"
    output_folder = Path("zipline_bundle") / output_folder
    output_folder = Path(output_folder)
    clear_directory(output_folder)

    with pd.HDFStore(DATA_STORE, mode='r') as store:
        total_rows = store.get_storer(price_key).nrows

    for start in range(0, total_rows, chunk_size):
        stop = min(start + chunk_size, total_rows)
        process_and_save_chunk(DATA_STORE, price_key, start, stop, output_folder)
        gc.collect()  # 가비지 컬렉션 호출

---